In [ ]:
from pprint import pprint
import librosa
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor
from transformers.pipelines.automatic_speech_recognition import AutomaticSpeechRecognitionPipeline

torch.set_float32_matmul_precision("high")

In [3]:
torch.set_float32_matmul_precision("high")
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_id = "openai/whisper-large-v3-turbo"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True,
).eval().cuda()
model.generation_config.cache_implementation = "static"
model.generation_config.max_new_tokens = 324
model.forward = torch.compile(model.forward, mode="reduce-overhead", fullgraph=True)

In [ ]:
processor = AutoProcessor.from_pretrained(model_id)
pipe = AutomaticSpeechRecognitionPipeline(
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch.float16,
    stride_length_s=10/6,
    chunk_length_s=10,
    device=device,
)

In [12]:
audio, _ = librosa.load("/home/demontego/whisper/tests/test_data/output.wav", sr=16000)
input_features = processor(audio, sampling_rate=16000, return_tensors="pt").input_features
input_features = input_features.to(torch.float16).cuda()

In [8]:
generate_kwargs = {
    'max_new_tokens': 444,
    'num_beams': 3,
    'condition_on_prev_tokens': False,
    'compression_ratio_threshold': 1.35,
    'temperature': (0.0, 0.2, 0.4, 0.6, 0.8, 1.0),
    'logprob_threshold': -1.0,
}

In [ ]:
result = pipe(audio, generate_kwargs=generate_kwargs, return_timestamps=True)
pprint(result)